# Simple Wordle solver using frame work from openai query 
Modified it but open ai did a good job.
Class called wordlesolver made it possible

In [1]:
import collections

class WordleSolver:
    def __init__(self, wordlist):
        self.wordlist = wordlist
        self.word_frequency = collections.Counter("".join(wordlist))
        self.letters_in_position = {}
        self.letters_not_in_position = {}
        self.letters_out_of_position = []
        self.letters_not_in_word = []
        

    def solve(self, result, letters_not_in_word):
        """
        Solve Wordle given constraints.

        :param letters_in_position: Dict[int, str] with positions (0-4) and their exact matching letters.
        :param letters_out_of_position: Dict[str, List[int]] with letters and their wrong positions.
        :param letters_not_in_word: List[str] of letters not in the word.
        :return: Tuple of filtered words with likelihoods and probable letters with likelihoods.
        """
        letters_in_position={i:result[i] for i in range(len(result)) if result[i].isupper()}
        self.letters_in_position={**self.letters_in_position,**letters_in_position}
        letters_not_in_position={i:result[i] for i in range(len(result)) if result[i].islower()}
        self.letters_not_in_position={**self.letters_not_in_position,**letters_not_in_position}
        letters_out_of_position=[result[i] for i in range(len(result)) if result[i].islower()]
        self.letters_out_of_position.extend(letters_out_of_position)
        self.letters_out_of_position=list(set(self.letters_out_of_position))
        self.letters_not_in_word.extend([letters_not_in_word[i] for i in range(len(letters_not_in_word))])
        
        possible_words = []
        for word in self.wordlist:
            if not self._matches_constraints(word, letters_in_position, letters_out_of_position, letters_not_in_word):
                continue
            possible_words.append(word)

        self.word_likelihoods = self._calculate_likelihoods(possible_words)
        self.letter_likelihoods = self._letter_frequencies(possible_words)
        self.word2 = self._word2()

        return self.word_likelihoods, self.letter_likelihoods, self.word2

    def _matches_constraints(self, word, letters_in_position, letters_out_of_position, letters_not_in_word):
        if len([i for i in solver.letters_out_of_position if i not in word]) >0:
            return False
        if len([i for i,j in solver.letters_in_position.items() if word[i]!=j.lower() ])>0:
                return False
        if len([i for i,j in solver.letters_not_in_position.items() if word[i]==j ]):
            return False
        if len([i for i in solver.letters_not_in_word if i in word ])>0:
            return False
        return True

    def _calculate_likelihoods(self, words):
        total_words = len(words)
        word_likelihoods = {word: sum(self.word_frequency[letter] for letter in word) for word in words}
        return sorted(word_likelihoods.items(), key=lambda x: x[1], reverse=True)

    def _letter_frequencies(self, words):
        combined_letters = "".join(words)
        letter_counts = collections.Counter(combined_letters)
        total_letters = sum(letter_counts.values())
        letter_likelihoods = {letter: count / total_letters for letter, count in letter_counts.items()}
        return sorted(letter_likelihoods.items(), key=lambda x: x[1], reverse=True)


    def _word2(self):
        inplace=list(self.letters_in_position.values())
        inplace.extend(list(self.letters_not_in_position.values()))
        letter_likelihoods_2 = {i[0]:i[1] for i in self.letter_likelihoods if i[0] not in inplace} #inplace 
        word_likelihoods = {word: sum(letter_likelihoods_2[letter] for letter in set(list(word)) if letter in letter_likelihoods_2) for word in self.wordlist}
        return sorted(word_likelihoods.items(), key=lambda x: x[1], reverse=True)[:10]
        

## Create the solver class

In [2]:
import pandas as pd
words_df=pd.read_csv("wordle.csv",header=None)
# Example Wordle word list (minimal for demonstration)
word_list = words_df[0].to_list()
solver = WordleSolver(word_list)


### Call solver.solve to resolve
input:
-    guess:string Exact match letter in uppercase, letter present lowercase, miss period
-    missed:string Letters not present
Output:
-    possible words with score
-    likiehood of letter being present

In [5]:
words, letters, word2 = solver.solve("s..rE","ta")
words, letters, word2 = solver.solve("r..SE","in")
words, letters, word2 = solver.solve(".o.SE","hu")

#words, letters = solver.solve("RELAX","")


#words, letters = solver.solve("..O..","plnk")
print("Possible words with likelihoods:", words)
print("------------------------")
print("Probable letters with likelihoods:", letters)
print("------------------------")
print("Probable words with good:", word2)

Possible words with likelihoods: [('prose', 3922)]
------------------------
Probable letters with likelihoods: [('p', 0.2), ('r', 0.2), ('o', 0.2), ('s', 0.2), ('e', 0.2)]
------------------------
Probable words with good: [('copse', 0.6000000000000001), ('lapse', 0.6000000000000001), ('parse', 0.6000000000000001), ('paste', 0.6000000000000001), ('pause', 0.6000000000000001), ('pesky', 0.6000000000000001), ('pesto', 0.6000000000000001), ('phase', 0.6000000000000001), ('poesy', 0.6000000000000001), ('poise', 0.6000000000000001)]


# Competitive:

In [136]:
solver = WordleSolver(word_list)
current_word='stare'
result='.....'
while not result.isupper():
    print(current_word)
    guess=input("Enter response")
    bad_letters="".join(list(set(list(current_word)) -  set(list(guess.lower().replace('.','')))))
    words, letters = solver.solve(guess,bad_letters)
    print(words[0])

stare


Enter response ..are


('rarer', 4909)
stare


Enter response Ra.e.


('renal', 4405)
stare


...

('relay', 4255)
stare


Enter response RELA.


('relay', 4255)
stare


Enter response RELa.


IndexError: list index out of range

'st'